In [ ]:
import boto3

s3 = boto3.client('s3', region_name = 'us-east-2')

bucket_name = ""

In [7]:
response = s3.list_buckets()

In [8]:
response

{'ResponseMetadata': {'RequestId': '14HGNP04W67BV78T',
  'HostId': '615L7pHGgrv6NN5hpi/sR4rwLahJ40RPYAezNtsH0wdm0lp56GmBwl65jOrdAmagSzyU5bhYlJw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '615L7pHGgrv6NN5hpi/sR4rwLahJ40RPYAezNtsH0wdm0lp56GmBwl65jOrdAmagSzyU5bhYlJw=',
   'x-amz-request-id': '14HGNP04W67BV78T',
   'date': 'Thu, 24 Apr 2025 01:00:29 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [],
 'Owner': {'ID': 'c10c9cd630c21df91c1dcb39a3ca2f5a723113798db1b18d4715a4d4514a38c2'}}

In [13]:
def create_bucket(bucket_name):
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
    print("Bucket is created 'jack' ")


create_bucket(bucket_name)

Bucket is created 'jack' 


Upload files to S3 here....

In [14]:
import os

def upload_file(file_path, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_path)

    s3.upload_file(file_path, bucket_name, object_name)


In [16]:
upload_file('index.html', "someotherfile.html")

In [19]:
def list_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)
    for obj in response['Contents']:
        print(obj['Key'])

In [22]:
list_objects()

index.html
someotherfile.html


In [30]:
def download_file(object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))

    s3.download_file(bucket_name, object_name, file_path)

download_file("index.html", "aws_sentiment_program/index.html")

In [44]:
def upload_directory(directory_path, s3_prefix):
    for root, dirs, files in os.walk('aws_sentiment_program'):
        for file in files:
            file_path = os.path.join(root, file).replace('\\', '/')
            relpath = os.path.relpath(file_path, directory_path)
           
            s3_key = os.path.join(s3_prefix, relpath).replace('\\','/')
            s3.upload_file(file_path, bucket_name, s3_key)

            

upload_directory("aws_sentiment_program", "s3_data")

Download a batch at a time if nessecasry....



In [51]:
local_path = "s3_download"
s3_prefix = "s3_data"

def download_dir(local_path, s3_prefix):
    os.makedirs(local_path, exist_ok=True)
    paginator = s3.get_paginator("list_objects_v2")

    for result in paginator.paginate(Bucket=bucket_name, Prefix= "s3_prefix"):
        if "Contents" in result:
            for key in result["Contents"]:
                s3_key = key["Key"]

                local_file = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
                #os.makedirs(local_file, exist_ok=True)

                s3.download_file(bucket_name, s3_key, local_file)


In [52]:
download_dir(local_path, s3_prefix)

Deleta all files in our bucket in AWS....

In [55]:
def delete_files():
    response = s3.list_objects_v2(Bucket=bucket_name)

    if "Contents" in response:
        for obj in response["Contents"]:
            s3.delete_object(Bucket=bucket_name, Key=obj["Key"])

delete_files()

In [1]:
s3.delete_bucket(Bucket=bucket_name)

NameError: name 's3' is not defined